In [1]:
## Import the relevant packages required
import re
import requests
import time
from requests.exceptions import RequestException
from bs4 import BeautifulSoup

In [ ]:
## Define your URL and user_agent
url = 'http://www.marketindex.com.au/rebalance-announcements'

response = requests.get(url)

## Returns a 403 error - this means the request was received but blocked.
print(response)

In [ ]:
## Define your URL and user_agent
url = 'http://www.marketindex.com.au/rebalance-announcements'

## Include a user_agent for authentication.
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

response = requests.get(url,headers={'User-Agent': user_agent})

## Returns 200 - success! 
print(response)

In [ ]:
## Parse the response with BS.
soup = BeautifulSoup(response.text,'html')

## Create an empty list to store download links
download_list = list()

## Find elements in the page that have the <a> tag (commonly used for links)
for each in soup.findAll('a'):
    print(each)

In [ ]:
## We only want the PDF links, so we include a parameter that the HTML link includes a '.pdf'. 
for each in soup.findAll('a'):
    if '.pdf' in each['href']:
        print(each['href'])

In [ ]:
## Need to include the full link to download, so we append the original website as well and store it in downloads.

for each in soup.findAll('a'):
    if '.pdf' in each['href']:
        ## I viewed the link to make sure the original appendice is correct. 
        download_url = 'http://www.marketindex.com.au/'+ each['href']
        download_list += [download_url]

## We have a list of PDF links to download now
print(download_list)

In [ ]:
## Loop through the list to download files
for each in download_list:
    r = requests.get(each,headers={'User-Agent': user_agent}, stream = True)
    
    ## Split the name based on '/' and name the files based on the URL link
    with open(each.split("/")[-1],"wb") as pdf: 
        for chunk in r.iter_content(): 
            # writing one chunk at a time to pdf file 
            if chunk: 
                ## Save the file in the same location as this script
                pdf.write(chunk)

Turning this into a class we can use on all websites.

In [46]:
## Import the relevant packages required
import re
import requests
import time
from requests.exceptions import RequestException
from bs4 import BeautifulSoup

class pdf_tool:
    def __init__(self,url):
        self.url = url
        self.user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        
        return None

    def checkurl(self):
        try:
            self.response = requests.get(self.url,headers={'User-Agent': self.user_agent})
            if '200' in str(self.response):
                return ("Successful connection.")
            else:
                return ("An error occurred in connecting.")
        except:
            return ("Invalid URL and/or request denied.")

    def find_link(self):
        soup = BeautifulSoup(self.response.text,'html')
        self.download_list = []
        for each in soup.findAll('a'):
            try:
                if '.pdf' in each['href']:
                    download_url = '/'.join(self.url.split('/')[0:3])+ each['href']
                    self.download_list += [download_url]
            except:
                return("There were no PDF links on this page.")
        if self.download_list == []:
            return ("There were no PDFs located on this page.")
        else:
            return ("List retrieved, call show_list() to view.")
    
    def show_list(self):
        if self.download_list == "" or self.download_list == []:
            return ("The list is empty.")
        else:
            return(self.download_list)
    
    def download_files(self):
        i = 0
        if self.download_list == "" or self.download_list == []:
            return ("The list is empty.")
        for each in self.download_list:
            r = requests.get(each,headers={'User-Agent': self.user_agent}, stream = True)
            i =+ 1
            
            ## Split the name based on '/' and name the files based on the URL link
            with open(each.split("/")[-1],"wb") as pdf: 
                for chunk in r.iter_content(chunk_size=1024): 
                    # writing one chunk at a time to pdf file 
                    if chunk: 
                        ## Save the file in the same location as this script
                        pdf.write(chunk)
            time.sleep(1)
        return(str(i) + " PDFs downloaded.")
    
    ## fastboi function to go fast boi
    def fastboi(self):
        self.checkurl()
        self.find_link()
        self.download_files()

In [49]:
## Success!

a = pdf_tool('http://www.marketindex.com.au/rebalance-announcements')
a.checkurl()
a.find_link()
a.show_list()
a.download_files()

ConnectionError: HTTPSConnectionPool(host='asic.gov.auhttps', port=443): Max retries exceeded with url: //download.asic.gov.au/media/5447183/regdocstable-july-dec-2019-forweb-2.pdf (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10b599f90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [48]:
a = pdf_tool('http://www.marketindex.com.au/rebalance-announcements')
a.fastboi()